In [3]:
#@title 이직 여부 판단 데이터

import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv")


display(x_train.head())
display(y_train.head())

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,25298,city_138,0.836,Male,No relevent experience,Full time course,High School,NaN,5,100-500,Pvt Ltd,1,45
1,4241,city_160,0.920,Male,No relevent experience,Full time course,High School,NaN,5,NaN,NaN,1,17
2,24086,city_57,0.866,Male,No relevent experience,no_enrollment,Graduate,STEM,10,NaN,NaN,1,50
3,26773,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,>4,135
4,32325,city_143,0.740,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,17


,enrollee_id,target
0,25298,0.0
1,4241,1.0
2,24086,0.0
3,26773,0.0
4,32325,1.0


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score ,precision_score

drop_col = ['enrollee_id','city','company_type','experience']
x_train_drop = x_train.drop(columns = drop_col)
x_test_drop = x_test.drop(columns = drop_col)

x_train_dummies = pd.get_dummies(x_train_drop)
y = y_train['target'].astype('int')

x_test_dummies = pd.get_dummies(x_test_drop)
x_test_dummies = x_test_dummies[x_train_dummies.columns]

X_train, X_val, y_train, y_val = train_test_split(x_train_dummies, y, test_size=0.33, random_state=42)
rf = RandomForestClassifier(random_state =23)
rf.fit(X_train,y_train)

predict_train_label = rf.predict(X_train)
predict_train_proba = rf.predict_proba(X_train)[:,1]

predict_validation_label = rf.predict(X_val)
predict_validation_prob = rf.predict_proba(X_val)[:,1]

predict_test_label = rf.predict(x_test_dummies)
predict_test_proba = rf.predict_proba(x_test_dummies)[:,1]

In [5]:
#@title 서비스 이탈예측 데이터

import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv")


display(x_train.head())
display(y_train.head())

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15799217,Zetticci,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
1,15748986,Bischof,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
2,15722004,Hsiung,543,France,Female,31,4,138317.94,1,0,0,61843.73
3,15780966,Pritchard,709,France,Female,32,2,0.00,2,0,0,109681.29
4,15636731,Ts'ai,714,Germany,Female,36,1,101609.01,2,1,1,447.73


,CustomerId,Exited
0,15799217,0
1,15748986,0
2,15722004,0
3,15780966,0
4,15636731,0


In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = x_train.drop(["CustomerId", "Surname"], axis = 1)
y = y_train.drop("CustomerId", axis = 1)
X_test = x_test.drop("Surname", axis = 1)
X_test_cust = X_test.pop("CustomerId")

label = LabelEncoder()
X["Geography"] = label.fit_transform(X["Geography"])
X["Gender"] = label.fit_transform(X["Gender"])
X_test["Geography"] = label.fit_transform(X_test["Geography"])
X_test["Gender"] = label.fit_transform(X_test["Gender"])

X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size = 0.3, random_state = 123)

lr = LogisticRegression()
lr.fit(X_tr, y_tr)
pred = lr.predict(X_val)
print("Logistic Regression - ", accuracy_score(y_val, pred))

svm = SVC()
svm.fit(X_tr, y_tr)
pred = svm.predict(X_val)
print("SVM - ", accuracy_score(y_val, pred))

dt = DecisionTreeClassifier()
dt.fit(X_tr, y_tr)
pred = dt.predict(X_val)
print("Decision Tree - " , accuracy_score(y_val, pred))

rf = RandomForestClassifier()
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
print("Random Forest - " , accuracy_score(y_val, pred))

model = LogisticRegression()
model.fit(X, y)
prediction = model.predict(X_test)

result = pd.DataFrame({"CustomerId":X_test_cust, "Exited":prediction})
result.to_csv("result.csv", index = False)

pd.read_csv("result.csv")

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Logistic Regression -  0.7912820512820513
SVM -  0.7948717948717948
Decision Tree -  0.781025641025641


<ipython-input-9-838f43109531>:40: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr, y_tr)


Random Forest -  0.8523076923076923


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,CustomerId,Exited
0,15601012,0
1,15734762,0
2,15586757,0
3,15590888,0
4,15726087,0
...,...,...
3496,15733966,1
3497,15669994,0
3498,15712403,0
3499,15643819,0


In [10]:
#@title 정시 배송 여부 판단

import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")


display(x_train.head())
display(y_train.head())

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1


In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

X = x_train.drop("ID", axis = 1)
y = y_train.drop("ID", axis = 1)
x_test_id = x_test.pop("ID")

label = LabelEncoder()
X["Warehouse_block"] = label.fit_transform(X["Warehouse_block"])
X["Mode_of_Shipment"] = label.fit_transform(X["Mode_of_Shipment"])
X["Product_importance"] = label.fit_transform(X["Product_importance"])
X["Gender"] = label.fit_transform(X["Gender"])
X["Customer_care_calls"] = label.fit_transform(X["Customer_care_calls"])
x_test["Warehouse_block"] = label.fit_transform(x_test["Warehouse_block"])
x_test["Mode_of_Shipment"] = label.fit_transform(x_test["Mode_of_Shipment"])
x_test["Product_importance"] = label.fit_transform(x_test["Product_importance"])
x_test["Gender"] = label.fit_transform(x_test["Gender"])
x_test["Customer_care_calls"] = label.fit_transform(x_test["Customer_care_calls"])

X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size = 0.25, random_state = 2023)

nb = GaussianNB()
nb.fit(X_tr, y_tr)
pred = nb.predict(X_val)
print("Naive Bayes -", accuracy_score(y_val, pred))

lr = LogisticRegression()
lr.fit(X_tr, y_tr)
pred = lr.predict(X_val)
print("Logistic Regression -", accuracy_score(y_val, pred))

svm = SVC()
svm.fit(X_tr, y_tr)
pred = svm.predict(X_val)
print("SVM -", accuracy_score(y_val, pred))

dt = DecisionTreeClassifier()
dt.fit(X_tr, y_tr)
pred = dt.predict(X_val)
print("Decision Tree -", accuracy_score(y_val, pred))

rf = RandomForestClassifier()
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
print("Random Forest -", accuracy_score(y_val, pred))

xgb = XGBClassifier()
xgb.fit(X_tr, y_tr)
pred = xgb.predict(X_val)
print("XGBoost -", accuracy_score(y_val, pred))

xgb2 = XGBClassifier(colsample_bytree=1, learning_rate=0.3, max_depth=6, min_child_weight=1, n_estimators=500)
xgb2.fit(X_tr, y_tr)
pred = xgb2.predict(X_val)
print("XGBoost2 -", accuracy_score(y_val, pred))

model = XGBClassifier()
model.fit(X, y)
prediction = model.predict(x_test)

result = pd.DataFrame({"ID":x_test_id, "Reached.on.Time_Y.N":prediction})
result.to_csv("result.csv", index = False)

pd.read_csv("result.csv")

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Naive Bayes - 0.6496969696969697
Logistic Regression - 0.6236363636363637
SVM - 0.6666666666666666
Decision Tree - 0.6436363636363637


<ipython-input-11-7eaf8ff57462>:52: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr, y_tr)


Random Forest - 0.649090909090909
XGBoost - 0.6321212121212121
XGBoost2 - 0.6351515151515151


,ID,Reached.on.Time_Y.N
0,6811,0
1,4320,1
2,5732,0
3,7429,1
4,2191,1
...,...,...
4396,2610,1
4397,3406,0
4398,10395,0
4399,3646,0


In [13]:
#@title 성인 건강검진 데이터

import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),...,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,구강검진수검여부,치아우식증유무,치석
0,0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,...,126.0,12.9,1.0,0.7,18.0,19.0,27.0,Y,0.0,Y
1,1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,...,127.0,12.7,1.0,0.6,22.0,19.0,18.0,Y,0.0,Y
2,2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,...,151.0,15.8,1.0,1.0,21.0,16.0,22.0,Y,0.0,N
3,3,M,40,165,70,88.0,1.5,1.5,1.0,1.0,...,226.0,14.7,1.0,1.0,19.0,26.0,18.0,Y,0.0,Y
4,4,F,40,155,60,86.0,1.0,1.0,1.0,1.0,...,107.0,12.5,1.0,0.6,16.0,14.0,22.0,Y,0.0,N


,ID,흡연상태
0,0,0
1,1,0
2,2,1
3,3,0
4,4,0


In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

X = x_train.drop("ID", axis = 1)
y = y_train.drop("ID", axis = 1)
x_test_id = x_test.pop("ID")

label = LabelEncoder()
X["성별코드"] = label.fit_transform(X["성별코드"])
X["구강검진수검여부"] = label.fit_transform(X["구강검진수검여부"])
X["치석"] = label.fit_transform(X["치석"])
x_test["성별코드"] = label.fit_transform(x_test["성별코드"])
x_test["구강검진수검여부"] = label.fit_transform(x_test["구강검진수검여부"])
x_test["치석"] = label.fit_transform(x_test["치석"])

X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size = 0.25, random_state = 123)

nb = GaussianNB()
nb.fit(X_tr, y_tr)
pred = nb.predict(X_val)
print("Naive Bayes -", accuracy_score(y_val, pred))

lr = LogisticRegression(max_iter = 300)
lr.fit(X_tr, y_tr)
pred = lr.predict(X_val)
print("Logistic Regression -", accuracy_score(y_val, pred))

dt = DecisionTreeClassifier(max_depth = 7)
dt.fit(X_tr, y_tr)
pred = dt.predict(X_val)
print("Decision Tree -", accuracy_score(y_val, pred))

rf = RandomForestClassifier(max_depth = 15, n_estimators = 500)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
print("Random Forest -", accuracy_score(y_val, pred))

xgb = XGBClassifier()
xgb.fit(X_tr, y_tr)
pred = xgb.predict(X_val)
print("XGBoost -", accuracy_score(y_val, pred))

model = RandomForestClassifier(max_depth = 15, n_estimators = 500)
model.fit(X, y)
prediction = model.predict(x_test)

result = pd.DataFrame({"ID":x_test_id, "흡연상태":prediction})
result.to_csv("result.csv", index = False)

pd.read_csv("result.csv")

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Naive Bayes - 0.7036538288894874


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression - 0.7364215818296077
Decision Tree - 0.7410898644402549


<ipython-input-14-e6a9e6b95dcf>:42: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr, y_tr)


Random Forest - 0.7633539815064189
XGBoost - 0.7566208815872161


<ipython-input-14-e6a9e6b95dcf>:52: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)


,ID,흡연상태
0,8,1
1,17,0
2,20,1
3,24,0
4,25,0
...,...,...
11134,55676,0
11135,55681,0
11136,55683,0
11137,55684,0
